# BEFORE YOU START, CHANGE THE RUNTIME TO REQUEST A T4 GPU

In [ ]:
# Installs
# You will need at a minimum the following packages. Feel free to install
# additional ones as needed
!pip install google-generativeai
!pip install datasets
!pip install -U bitsandbytes
!pip install transformers
!pip install -U peft
!pip install -U "huggingface_hub[cli]"
!pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.2 MB/s eta 0:00:00
   ━━━━

In [ ]:
import google.generativeai as genai
from datasets import Dataset, DatasetDict
import pandas as pd
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, \
    BitsAndBytesConfig, TrainingArguments, pipeline, logging
import torch
from trl import SFTTrainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import required libraries
import os
import json
import numpy as np
import google.generativeai as genai
from google.colab import files
from IPython.display import display, HTML

# Configure the Gemini API
geminiApiKey = "Api_key"
genai.configure(api_key=geminiApiKey)
model = genai.GenerativeModel('gemini-pro')
cfg = genai.types.GenerationConfig(max_output_tokens=4000)

def split_data(qa_pairs, train_ratio=0.7):
    np.random.shuffle(qa_pairs)
    train_size = int(len(qa_pairs) * train_ratio)
    train_data = qa_pairs[:train_size]
    validation_data = qa_pairs[train_size:]
    return train_data, validation_data

# Function to upload and read a file
def upload_and_read_file():


# Function to generate Q&A pairs
def generate_qa(content):
    prompt = f"""
    Please generate 20 additional question-answer pairs that are similar in style and content to the following question-answer pairs from the text file. In the text file, Q: is followed by a question, and A: is followed by the corresponding answer.

    Create these new pairs along with the original ones from the text file. Format each question and answer as a separate Q&A pair, separated by double new lines. Ensure that there are more than 30 question-answer pairs in total (including both the original and new ones).

    {content.strip()}

    Format the output as:
    Q: [Question]
    A: [Answer]

    Separate each Q&A pair with a blank line.
    """

    response = model.generate_content(prompt, generation_config=cfg)
    q_and_a_pairs = response.text.split("\n\n")  # Assuming pairs are separated by double newlines

    qa_list = []

    for pair in q_and_a_pairs:
        if "Q:" in pair and "A:" in pair:
            question = pair.split("Q:")[1].split("A:")[0].strip()
            answer = pair.split("A:")[1].strip()
            qa_list.append({"question": question, "answer": answer})

    return qa_list

# Main execution
print("Please upload your input text file.")
file_content = upload_and_read_file()

qa_pairs = generate_qa(file_content)

# Split the data into training and validation sets
train_data, validation_data = split_data(qa_pairs);

# Save training data to JSON file
train_filename = 'train_qa_pairs.json'
with open(train_filename, 'w', encoding='utf-8') as json_file:
    json.dump(train_data, json_file, ensure_ascii=False, indent=2)

# Save validation data to JSON file
validation_filename = 'validation_qa_pairs.json'
with open(validation_filename, 'w', encoding='utf-8') as json_file:
    json.dump(validation_data, json_file, ensure_ascii=False, indent=2)

print(f"Training data saved to {train_filename}")
print(f"Validation data saved to {validation_filename}")

# Display sample Q&A pairs from training set
display(HTML("<h3>Sample Training Q&A Pairs:</h3>"))
for pair in train_data[:3]:  # Display first 3 pairs
    display(HTML(f"<b>Q:</b> {pair['question']}<br><b>A:</b> {pair['answer']}<br><br>"))

# Display sample Q&A pairs from validation set
display(HTML("<h3>Sample Validation Q&A Pairs:</h3>"))
for pair in validation_data[:2]:  # Display first 2 pairs
    display(HTML(f"<b>Q:</b> {pair['question']}<br><b>A:</b> {pair['answer']}<br><br>"))

# Option to download the JSON files
files.download(train_filename)
files.download(validation_filename)

Please upload your input text file.


StopIteration: 

In [ ]:
import json
import os, re
from google.colab import files

result = []
directory = "/content/drive/MyDrive/FDATA/val"

for filename in os.listdir(directory):
    if filename.endswith(".json"):
        file_path = os.path.join(directory, filename)
        with open(file_path, "r") as infile:
            result.append(json.load(infile))

flattened_data = [item for sublist in result for item in sublist]

for item in flattened_data:
        if "Question" in item:
            item["Question"] = re.sub(r'^\s*\*\*Question:\*\*\s*', '', item["Question"]).strip()
        if "Answer" in item:
            item["Answer"] = item["Answer"].lstrip('*').strip()

# Save the merged result to a new JSON file
merged_file_path = "merged_file.json"
with open(merged_file_path, "w") as outfile:
    json.dump(flattened_data, outfile, indent=4)

# Download the merged file
files.download(merged_file_path)

print(f"Merged {len(result)} JSON files into '{merged_file_path}'")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Merged 6 JSON files into 'merged_file.json'


In [ ]:
print(len(flattened_data))

160


In [ ]:
import json
from datasets import Dataset, DatasetDict
import pandas as pd

file_path = '/content/drive/MyDrive/qa_pairs(8).json'
# Load question-answer pairs from JSON file
def load_qa_pairs(file_path):
    with open('/content/drive/MyDrive/qa_pairs(8).json', 'r') as file:
        qa_pairs = json.load(file)
    return qa_pairs

# Assuming your JSON file is named 'qa_pairs.json'
qa_pairs = load_qa_pairs('qa_pairs.json')

# Split the data into train and validation sets (80-20 split)
split_index = int(len(qa_pairs) * 0.8)
qa_pairs_train = qa_pairs[:split_index]
qa_pairs_val = qa_pairs[split_index:]

# Create datasets
dataset_train = Dataset.from_pandas(pd.DataFrame(qa_pairs_train))
dataset_val = Dataset.from_pandas(pd.DataFrame(qa_pairs_val))

# Create DatasetDict
dataset = DatasetDict({
    "train": dataset_train,
    "val": dataset_val
})

# Print dataset info
print(dataset)

# Print the first example from the training set
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 11
    })
    val: Dataset({
        features: ['question', 'answer'],
        num_rows: 3
    })
})
{'question': 'Provide an expression that sets `A` to be a 3 * 2 numpy array (3 rows by 2 columns), containing any values you wish.', 'answer': '```python\nA_111 = np.array([[10,20],[40,50],[70,80]])\n```'}


In [ ]:
import json
import numpy as np

In [ ]:
file_path1 = '/content/drive/MyDrive/f/ntraining_data.json'
file_path2 = '/content/drive/MyDrive/f/nvalidation_data.json'

# Load question-answer pairs from JSON file
def load_qa_pairs(file_path):
    with open(file_path, 'r') as file:
        qa_pairs = json.load(file)
    return qa_pairs

qa_pairs_train = load_qa_pairs(file_path1)
qa_pairs_val = load_qa_pairs(file_path2)

np.random.shuffle(qa_pairs_train)
np.random.shuffle(qa_pairs_val)

dataset_train = Dataset.from_pandas(pd.DataFrame(qa_pairs_train[:100]))
dataset_val = Dataset.from_pandas(pd.DataFrame(qa_pairs_val[:30]))

# Create DatasetDict
dataset = DatasetDict({
    "train": dataset_train,
    "val": dataset_val
})

# Print dataset info
print(dataset)

# Print the first example from the training set
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 100
    })
    val: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 30
    })
})
{'Question': "What does the text imply about the nature of the set's elements?", 'Answer': 'The elements are independent except for sharing a common probability distribution.'}


In [ ]:
dataset["train"][0]

{'Question': "What does the text imply about the nature of the set's elements?",
 'Answer': 'The elements are independent except for sharing a common probability distribution.'}

In [ ]:
print(dataset["train"].features)

{'Question': Value(dtype='string', id=None), 'Answer': Value(dtype='string', id=None)}


In [ ]:
def format_data(example):
    return {
        "text": f"Question: {example['Question']}\n\nAnswer: {example['Answer']}"
    }

dataset["train"] = dataset["train"].map(format_data)
dataset["val"] = dataset["val"].map(format_data)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [ ]:
print(dataset["train"])

Dataset({
    features: ['Question', 'Answer', 'text'],
    num_rows: 100
})


In [ ]:
from huggingface_hub import login

# Replace 'your_access_token' with the actual token you generated
login(token="your_access_token")

In [ ]:
# Load the model -- Skeleton
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
#    llm_int8_enable_fp32_cpu_offload=True
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
# Tokenize the data
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

('<s>', '</s>')

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    return {
        "input_ids": tokenized["input_ids"].squeeze().to(torch.long),
        "attention_mask": tokenized["attention_mask"].squeeze().to(torch.long)
    }

# Apply to both train and test datasets
tokenized_train = Dataset.from_dict(tokenize_function(dataset["train"]))
tokenized_eval = Dataset.from_dict(tokenize_function(dataset["val"]))

In [ ]:
print(tokenized_train[0])

{'input_ids': [1, 22478, 28747, 1824, 1235, 272, 2245, 25385, 684, 272, 4735, 302, 272, 808, 28742, 28713, 5176, 28804, 13, 13, 2820, 16981, 28747, 415, 5176, 460, 7126, 3741, 354, 10681, 264, 3298, 10966, 6023, 28723, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

In [ ]:
# LoRA config -- Skeleton
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

In [ ]:
# Hyperparameters -- Skeleton
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=50,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    gradient_accumulation_steps=4,
    fp16=True,
    learning_rate=2e-4,
    remove_unused_columns=False,
    push_to_hub=False,
)

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla T4


In [ ]:
# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=512,
    dataset_text_field="text",
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
0,3.389900,3.516003
1,3.372700,3.426169
2,3.198800,3.252023
4,2.645300,2.644043
5,2.407700,2.412679
6,2.218300,2.248405
8,1.835600,1.926566
9,1.661900,1.836960
10,1.528100,1.785978
12,1.248000,1.759054


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

KeyboardInterrupt: 

In [ ]:
# Save the model
save_path = "/content/drive/MyDrive"
trainer.model.save_pretrained("save_path")
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k

In [ ]:
# Run the model locally
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, truncation=True)
def build_prompt(question):
  prompt = f"<s>[INST]@ESE577. {question}. [/INST]"
  return prompt

while True:
  question = input("Enter your ESE577-related question (hit Enter to exit): ").strip()
  if not question:
    break
  prompt = build_prompt(question)
  answer = pipe(prompt)
  print(answer[0]["generated_text"])
  print()


Enter your ESE577-related question (hit Enter to exit): what is mentioned in section 1.3
<s>[INST]@ESE577. what is mentioned in section 1.3. [/INST]1.3 Outlines several applications of gradient descent.

The text mentions several applications of gradient descent, including optimization, regression, classification, and solving differential equations.

Enter your ESE577-related question (hit Enter to exit): which chapter is section 1.3 from
<s>[INST]@ESE577. which chapter is section 1.3 from. [/INST]1.3 is not from a specific chapter, but rather refers to section 1.3 of a text or document.

Enter your ESE577-related question (hit Enter to exit): what does ESE577 course tell us about gradient descent mention specific details
<s>[INST]@ESE577. what does ESE577 course tell us about gradient descent mention specific details. [/INST]The ESE577 course describes gradient descent as an iterative process for finding local optima of a function, focusing on its application to regression and classif

KeyboardInterrupt: Interrupted by user